In [1]:
import numpy as np
from scipy.special import fresnel, erf
import matplotlib.pyplot as plt

In [2]:
def get_SC(x):
    ip = 1j + 1
    im = 1 - 1j
    arg_p = ip * np.sqrt(np.pi) * x * 0.5
    arg_m = im * np.sqrt(np.pi) * x * 0.5
    S = (ip/4) * (erf(arg_p) - 1j * erf(arg_m))
    C = (im/4) * (erf(arg_p) + 1j * erf(arg_m))
    return S, C

In [3]:
chi1 = 1
dtau = 1
v0   = 1
v1   = 2
N    = 1e3
chi2 = np.linspace(-1e-3, 1e-3, int(N))

In [4]:
# Fresenel
phi_p = (dtau**2 / 4) * chi2 + (dtau / 2) * chi1
phi_m = (dtau**2 / 4) * chi2 - (dtau / 2) * chi1
psi_base = np.sqrt(2 * np.pi / chi2, dtype = 'complex128') * (2 * chi2 * v0 - chi1 * v1)
psi_p    = psi_base * np.cos(chi1**2 / (4 * chi2))
psi_m    = psi_base * np.sin(chi1**2 / (4 * chi2))
theta_p = (chi1 + chi2 * dtau) / np.sqrt(2 * np.pi * chi2, dtype = 'complex128')
theta_m = (chi1 - chi2 * dtau) / np.sqrt(2 * np.pi * chi2, dtype = 'complex128')
Sp, Cp  = get_SC(theta_p)
Sm, Cm  = get_SC(theta_m)
Rf      = (1 / (4 * chi2)) * (psi_p * (Cp-Cm) + psi_m * (Sp - Sm) + 2 * v1 * (np.sin(phi_p) - np.sin(phi_m)))
If      = (1 / (4 * chi2)) * (psi_p * (Sp-Sm) - psi_m * (Cp - Cm) - 2 * v1 * (np.cos(phi_p) - np.cos(phi_m)))

In [5]:
# Taylor
arg = chi1 * dtau * 0.5
sinc = np.sinc(arg / np.pi)
I0   = sinc * dtau
I1   = (dtau / chi1) * (sinc - np.cos(arg))
I2   = (dtau**3 / 4) * sinc - (2 * I1 / chi1)
Rt   = np.zeros(len(chi2)) + v0 * I0
It   = v1 * I1 + chi2 * v0 * I2

In [6]:
print(any(np.imag(Rf)))
print(any(np.imag(If)))

False
False


In [7]:
fres = np.real(Rf) + 1j * np.real(If)
tay  = Rt + 1j * It